## 1(2)

In [3]:
!pip install pulp
!pip install pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 33.4 MB/s eta 0:00:00


In [1]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

model = LpProblem(name="maximize_profit", sense=LpMaximize)


plants = [1, 2, 3]
distribution_centers = list(range(1, 13))

capacity = {1: 120, 2: 90, 3: 100}
cost = {1: 8, 2: 7, 3: 9}

demand = {1: 20, 2: 30, 3: 40, 4: 35, 5: 15, 6: 19, 7: 25, 8: 20, 9: 18, 10: 12, 11: 30, 12: 40}

transport_cost = {
    (1,1): 2, (1,2): 3, (1,3): 2, (1,4): 3, (1,5): 5, (1,6): 4.4, (1,7): 3.2, (1,8): 3, (1,9): 1.5, (1,10): 3, (1,11): 6.0, (1,12): 1.0,
    (2,1): 1.9, (2,2): 3, (2,3): 2.4, (2,4): 2.4, (2,5): 2, (2,6): 4.1, (2,7): 2.8, (2,8): 3, (2,9): 1.7, (2,10): 2, (2,11): 1.0, (2,12): 4.5,
    (3,1): 3.1, (3,2): 1.8, (3,3): 2.8, (3,4): 3.4, (3,5): 2, (3,6): 2.9, (3,7): 2.2, (3,8): 3.2, (3,9): 2, (3,10): 2.8, (3,11): 4.5, (3,12): 6.0
}

x = {(i, j): LpVariable(name=f"x_{i}_{j}", lowBound=0) for i in plants for j in distribution_centers}

model += lpSum((20 - cost[i] - transport_cost[i, j]) * x[i, j] for i in plants for j in distribution_centers), "Total_Profit"

for i in plants:
    model += lpSum(x[i, j] for j in distribution_centers) <= capacity[i], f"Supply_Limit_{i}"

for j in distribution_centers:
    model += lpSum(x[i, j] for i in plants) == demand[j], f"Demand_Fulfillment_{j}"

model.solve()

print("VALUES are in 10000s")
print("\nOptimal Production and Transportation Plan:")
for i in plants:
    for j in distribution_centers:
        if x[i, j].value() > 0:
            print(f"Send {x[i, j].value()} units from Plant {i} to DC {j}")

print(f"\nMaximum Profit: Rs. {model.objective.value()}")


VALUES are in 10000s

Optimal Production and Transportation Plan:
Send 7.0 units from Plant 1 to DC 1
Send 40.0 units from Plant 1 to DC 3
Send 15.0 units from Plant 1 to DC 8
Send 18.0 units from Plant 1 to DC 9
Send 40.0 units from Plant 1 to DC 12
Send 13.0 units from Plant 2 to DC 1
Send 35.0 units from Plant 2 to DC 4
Send 12.0 units from Plant 2 to DC 10
Send 30.0 units from Plant 2 to DC 11
Send 30.0 units from Plant 3 to DC 2
Send 15.0 units from Plant 3 to DC 5
Send 19.0 units from Plant 3 to DC 6
Send 25.0 units from Plant 3 to DC 7
Send 5.0 units from Plant 3 to DC 8

Maximum Profit: Rs. 3065.2000000000003
